In [ ]:
# Train the CNN Emotion Regression Model with MIDI data#
# Last editted by Pu Zeng, 19/10/2023 #
# The result is really bad because we only use a small portion of the dataset

# Install the library needed

In [ ]:
!git clone https://github.com/Pu-Zeng/696-Milestone.git

Cloning into '696-Milestone'...
remote: Enumerating objects: 3018, done.
remote: Counting objects: 100% (3018/3018), done.
remote: Compressing objects: 100% (2531/2531), done.
remote: Total 3018 (delta 43), reused 3008 (delta 33), pack-reused 0
Receiving objects: 100% (3018/3018), 32.22 MiB | 24.42 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [ ]:
!pip install langdetect
!pip install pretty_midi
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=b2dd0480552371ede141d6541bdbbc6effa08e00c2e6b69eb0b432f50957ec2b
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 5.9 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=57c6f1e6dab1469b69a166efeeed45a96b99bd476ceab218f0e8ca97542eec97
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.5 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
# import mido
import string
import numpy as np
# from utilis import get_pianoroll_data
import pickle
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
# import mido
import string
import numpy as np
# from utilis import get_pianoroll_data
import pickle
from sklearn import preprocessing
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

from torch.utils.data import TensorDataset


# Data Preprocess

In [ ]:
# Use our python script to preprocess the midi files into matrices and lyrics
!mkdir Processed_Data
!python /content/696-Milestone/Data_Preprocessing/Emotion_MIDI2.py /content/696-Milestone/Toy_Dataset/Lakh/ ./Processed_Data/

In [ ]:
# Use our python script to generate the labels using Flan-T5
!python /content/696-Milestone/Data_Preprocessing/Emotion_Label.py ./Processed_Data/ ./Processed_Data/

In [ ]:
# Open the results of the last two cells
file=open("./Processed_Data/lakh_emotion.bin","rb")
music_data = pickle.load(file) #保存list到文件
file.close()

In [ ]:
# Show the labels
np.unique([x[1] for x in music_data])

In [ ]:
# Selected the most common one word labels
from tqdm import tqdm
X = []
y = []
chosen_label = ['sadness', 'happy', 'love', 'joy', 'negative', 'positive', 'anger',
       'awe', 'emotional', 'hopeful', 'angry', 'excited', 'fear', 'hurt',
       'fearful']
# for i in tqdm(range(1,10)):
file=open("./Processed_Data/lakh_emotion.bin","rb")
music_data = pickle.load(file) #保存list到文件
file.close()
for m in music_data:
    if m[1] in chosen_label:
        X.append(m[0])
        if m[1] == 'anger':
            m[1] = 'angry'
        if m[1] == 'fearful':
            m[1] = 'fear'
        if m[1] == 'joy':
            m[1] = 'happy'
        y.append(m[1])
X = np.array(X)

In [ ]:
# The dataset is just a toy dataset, so there are far less labels than the whole Lakh dataset, we need to drop those will few samples
y = np.array(y)
X_new = []
y_new = []
for l in np.unique(y):
    n = (y==l).sum()
    if n>=6:
        X_new += list(X[y==l])
        y_new += list(y[y==l])
X = X_new
y = y_new

In [ ]:
# Transform labels into one-hot variable
from sklearn import preprocessing
import torch

le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

label_dict = {}
for cl in le.classes_:
    label_dict.update({cl:le.transform([cl])[0]})

In [ ]:
num_labels = len(np.unique(y))

# Train the CNN model

In [ ]:
#Define CNN model
class CNNModel(nn.Module):
    def __init__(self, hiddenSize, outChannels, dropoutRate, activate):
        super().__init__()
        self.outChannels = outChannels
        self.activate = nn.Sigmoid() if activate == "Sigmoid" else nn.ReLU()
        self.conv1 = nn.Conv2d(1, 24, (10,1))
        self.pool = nn.MaxPool2d((2, 1))
        self.conv2 = nn.Conv2d(24, 48, (10,1))
        self.conv3 = nn.Conv2d(48, 96, (10,1))
        self.conv4 = nn.Conv2d(96, 192, (10,1))
        self.conv5 = nn.Conv2d(192, 384, (5,2))
        self.conv6 = nn.Conv2d(384, 192, (5,2))
        self.dense1 = nn.Linear(48384, hiddenSize)
        self.dropout = nn.Dropout(dropoutRate)
        self.dense2 = nn.Linear(hiddenSize, num_labels)

    def forward(self, x):
        x = self.pool(self.activate(self.conv1(x)))
        x = self.pool(self.activate(self.conv2(x)))
        # print(x.shape)
        x = self.dropout(self.pool(self.activate(self.conv3(x))))
        x = self.pool(self.activate(self.conv4(x)))
        x = self.pool(self.activate(self.conv5(x)))
        x = self.pool(self.activate(self.conv6(x)))
        # print(x.shape)
        x = x.view(-1, 48384)
        x = self.dropout(self.activate(self.dense1(x)))
        return self.dense2(x)

# Number of neurons in the first fully-connected layer
hiddenSize = 4096
# Number of feature filters in second convolutional layer
numFilters = 25
# Dropout rate
dropoutRate = 0.2
# Activation function
activation = "ReLU"
# Learning rate
learningRate = 0.005
# Momentum for SGD optimizer
momentum = 0.9
# Number of training epochs
numEpochs = 20

In [ ]:
#Define Trian and Validation function

from tqdm import tqdm
def train_epoch(cnn,device,dataloader,loss_fn,optimizer):
    cnn.train()
    cnnRunningLoss = 0
    total = 0
    R2 = 0
    cnnCorrect=0
    total1=0
    for i, (inputs, labels) in enumerate(dataloader, 0):
        optimizer.zero_grad()
        inputs = inputs.to(device).reshape(-1,1,512,128)

        labels = labels.to(device)
        # Forward propagation
        cnnOutputs = cnn(inputs)
        # print(cnnOutputs.shape)
        l2_lambda = 0.001
        l2_reg = torch.tensor(0.).to(device)
        for param in cnn.parameters():
            l2_reg += torch.norm(param)
        # Backpropagation
        cnnLoss = criterion(cnnOutputs, labels)+l2_reg*l2_lambda
        cnnLoss.backward()
        # Gradient update
        optimizer.step()
        total += 1
        total1+=labels.size(0)
        cnnRunningLoss += cnnLoss.item()
        _, cnnPredicted = torch.max(cnnOutputs.data, 1)
        cnnCorrect += (cnnPredicted == labels).sum().item()
    return cnnRunningLoss/total, cnnCorrect/total1

def valid_epoch(cnn,device,dataloader,loss_fn):
    cnn.eval()
    totalLoss = 0
    total = 0
    total1 = 0
    R2 = 0
    cnnLoss = 0
    cnnCorrect=0
    for inputs, labels in dataloader:
        inputs = inputs.to(device).reshape(-1,1,512,128)
        labels = labels.to(device)
        cnnOutputs = cnn(inputs)
        cnnLoss = loss_fn(cnnOutputs, labels)
        _, cnnPredicted = torch.max(cnnOutputs.data, 1)
        total += labels.size(0)
        total1 +=1
        totalLoss += cnnLoss.item()
        cnnCorrect += (cnnPredicted == labels).sum().item()
    accuracy = cnnCorrect / total * 100
    cnn.train()
    return totalLoss/total1, accuracy

In [ ]:
# Train the model
import sklearn
from sklearn.model_selection import StratifiedKFold
X_train_v, X_test, y_train_v, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) # First Split the dataset and use the training set for 5-fold validation

X_train_v = torch.as_tensor(X_train_v, dtype=torch.float) # an alternative to torch.from_numpy
y_train_v = torch.as_tensor(y_train_v, dtype=torch.float)
X_test = torch.as_tensor(X_test, dtype=torch.float)
y_test = torch.as_tensor(y_test, dtype=torch.float)
splits=StratifiedKFold(n_splits=5,shuffle=True,random_state=99)
train_dataset = TensorDataset(X_train_v, y_train_v)
test_dataset = TensorDataset(X_test, y_test)
batch_size=128
models = []
history = {'fold':[], 'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
for fold, (train_idx,val_idx) in enumerate(splits.split(X_train_v,y_train_v)):
    print('Fold {}'.format(fold + 1))
    t = pd.DataFrame(y_train_v[train_idx], columns = ['class']).reset_index()
    idx = []
    # Resample to avoid unbalanced dataset
    y_train = []
    for i in range(0,num_labels):
      idx += list(t[t['class']==i].sample(100,replace=True)['index'])
      y_train += [i]*100
    X_train = X_train_v[train_idx][idx]

    t = pd.DataFrame(y_train_v[val_idx], columns = ['class']).reset_index()
    idx = []
    y_val = []
    for i in range(0,num_labels):
      # print(i)
      if i in t['class'].values:
          idx += list(t[t['class']==i].sample(20,replace=True)['index'])
          y_val += [i]*20
    X_val =  X_train_v[val_idx][idx]

    X_train = torch.as_tensor(X_train, dtype=torch.float) # an alternative to torch.from_numpy
    y_train = torch.as_tensor(y_train, dtype=torch.float).type(torch.LongTensor)
    X_val = torch.as_tensor(X_val, dtype=torch.float)
    y_val = torch.as_tensor(y_val, dtype=torch.float).type(torch.LongTensor)

    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_val, y_val)

    from torch.utils.data import Dataset, DataLoader
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    model = CNNModel(hiddenSize, numFilters, dropoutRate, activation).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(list(model.parameters()), lr=learningRate, momentum=momentum)
    # optimizer = torch.optim.Adam(list(model.parameters()), lr=learningRate)
    best_test = -np.inf
    best_model = None

    for epoch in range(numEpochs):
        train_loss, train_R2=train_epoch(model,device,train_dataloader,criterion,optimizer)
        test_loss, test_R2=valid_epoch(model,device,test_dataloader,criterion)
        history['fold'].append(fold)
        history['train_loss'].append(train_loss)
        history['test_loss'].append(test_loss)
        history['train_acc'].append(train_R2)
        history['test_acc'].append(test_R2)
        if test_R2>best_test:
            test_test = test_R2
            best_model = model
        print('Train Loss: {}, Train_acc: {}, Test Loss: {}, Test acc: {}'.format(train_loss, train_R2, test_loss, test_R2))
    models.append([best_model])

# Evaluation

In [ ]:
# Caculate the average accuray
# X_train_v, X_test, y_train_v, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
t = pd.DataFrame(y_test, columns = ['class']).reset_index()
idx = []
y_test = []
for i in range(0,num_labels):
    if i in t['class'].values:
        idx += list(t[t['class']==i].sample(20,replace=True)['index'])
        y_test += [i]*20
X_test = np.array(X_test)
X_test =  X_test[idx]
X_test = torch.as_tensor(X_test, dtype=torch.float)
y_test = torch.as_tensor(y_test, dtype=torch.float).type(torch.LongTensor)
test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
acc=[]
for best_model in models:
    acc.append(valid_epoch(best_model[0],device,test_dataloader,criterion)[1])

In [ ]:
np.mean(acc)

In [ ]:
# Calculate the accuray of the ensemble model
with torch.no_grad():
  outputs = []
  for model in models:
    model[0].eval()
    model_output = []
    label_= []
    model[0].to('cpu')
    for inputs, labels in test_dataloader:
        inputs = inputs.to('cpu').reshape(-1,1,512,128)
        labels = labels.to('cpu')
        cnnOutputs = model[0](inputs)
        model_output.append(cnnOutputs)
        label_.append(labels)
    outputs.append(torch.vstack(model_output))
  shape = outputs[0].shape
  cnnresult = torch.vstack(outputs).reshape(-1,shape[0],shape[1]).mean(axis=0)
  labels = np.hstack(label_)
  _, cnnPredicted = torch.max(cnnresult.data, 1)
  cnnCorrect = (cnnPredicted.detach().numpy() == labels).sum().item()
  cnnCorrect/len(labels)*100

In [ ]:
# Draw the confusion matrix
from sklearn.metrics import confusion_matrix
confu_m = confusion_matrix(labels, cnnPredicted)
import seaborn as sns
sns.heatmap(confu_m,square=False)

In [ ]:
label_dict = {}
for cl in le.classes_:
    label_dict.update({cl:le.transform([cl])[0]})
label_dict

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
scores = pd.DataFrame(precision_recall_fscore_support(cnnPredicted, labels, labels = list(range(num_labels))),index=['precision','recall','f1','support'], columns = list(label_dict)[:num_labels])
scores